In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import os
import matplotlib.pyplot as plt
import string
import re
import logging
from gensim import corpora
from gensim.models import Phrases
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
customer_comment=[]


In [3]:
page_count = 0
while page_count < 11:
    
    url = "https://www.consumeraffairs.com/finance/royal_bank.html?page=%s" %(page_count)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    reviews=soup.find(id="reviews-container")
    
    comments=soup.find_all("div", class_="rvw-bd ca-txt-bd-2")
    for ct in comments:
        customer_comment.append(ct.find("p").get_text())
    
    page_count += 1   

In [4]:
RBC_Reviews = pd.DataFrame({
"Review":customer_comment
})


RBC_Reviews.head(20)

,Review
0,We met Ratna in June 2012 as we move to Calgar...
1,I have found their staff to be extremely helpf...
2,We opened two accounts with RBC. Jamie ** of t...
3,What a POS these PVQa are. This idiotic piece ...
4,Today I heard from the Manager. He was except...
5,Hi my name is Fidi and I have been a client wi...
6,Extremely rude people work on the other end of...
7,I've been a RBC client for nearly 20 years and...
8,"Wow, I have never dealt with a more unprofessi..."
9,Business owners stay away from this place. An ...


In [5]:
RBC_Reviews['Review'] = RBC_Reviews['Review'].str.replace('[^\w\s]','')
RBC_Reviews['Review'].head()

0    We met Ratna in June 2012 as we move to Calgar...
1    I have found their staff to be extremely helpf...
2    We opened two accounts with RBC Jamie  of the ...
3    What a POS these PVQa are This idiotic piece o...
4    Today I heard from the Manager  He was excepti...
Name: Review, dtype: object

In [6]:

stop = stopwords.words('english')
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
RBC_Reviews['Review'].head()

0    We met Ratna June 2012 move Calgary At time go...
1    I found staff extremely helpful Having worked ...
2    We opened two accounts RBC Jamie Salmon Arm br...
3    What POS PVQa This idiotic piece software deci...
4    Today I heard Manager He exceptionally good de...
Name: Review, dtype: object

In [10]:
freq = pd.Series(' '.join(RBC_Reviews['Review']).split()).value_counts()[:10]
freq

I          1527
RBC         415
account     326
bank        290
credit      171
money       146
The         138
They        133
years       132
told        120
dtype: int64

In [11]:
common_word=("I","The","They","rbc","account","bank","get","told","card")
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in common_word))
RBC_Reviews['Review'].head()

0    We met Ratna June 2012 move Calgary At time go...
1    found staff extremely helpful Having worked ma...
2    We opened two accounts RBC Jamie Salmon Arm br...
3    What POS PVQa This idiotic piece software deci...
4    Today heard Manager He exceptionally good deal...
Name: Review, dtype: object

In [12]:
least = pd.Series(' '.join(RBC_Reviews['Review']).split()).value_counts()[-10:]
least

nickel         1
rented         1
touch          1
young          1
Glendale       1
dead           1
tax            1
negotiation    1
surprise       1
demolition     1
dtype: int64

In [13]:
least = list(least.index)
RBC_Reviews['Review'] = RBC_Reviews['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in least))
RBC_Reviews['Review'].head()

0    We met Ratna June 2012 move Calgary At time go...
1    found staff extremely helpful Having worked ma...
2    We opened two accounts RBC Jamie Salmon Arm br...
3    What POS PVQa This idiotic piece software deci...
4    Today heard Manager He exceptionally good deal...
Name: Review, dtype: object

In [14]:

RBC_Reviews['tokens'] = RBC_Reviews['Review'].apply(word_tokenize) 
RBC_Reviews['tokens'].head()

0    [We, met, Ratna, June, 2012, move, Calgary, At...
1    [found, staff, extremely, helpful, Having, wor...
2    [We, opened, two, accounts, RBC, Jamie, Salmon...
3    [What, POS, PVQa, This, idiotic, piece, softwa...
4    [Today, heard, Manager, He, exceptionally, goo...
Name: tokens, dtype: object

In [15]:

porter_stemmer = PorterStemmer()
RBC_Reviews['stem']=RBC_Reviews['tokens'].apply(lambda x : [porter_stemmer.stem(y) for y in x])
RBC_Reviews['stem'].head()

0    [We, met, ratna, june, 2012, move, calgari, At...
1    [found, staff, extrem, help, have, work, mani,...
2    [We, open, two, account, rbc, jami, salmon, ar...
3    [what, po, pvqa, thi, idiot, piec, softwar, de...
4    [today, heard, manag, He, except, good, dealt,...
Name: stem, dtype: object

In [16]:
unlist_comments = [item for items in RBC_Reviews['stem'] for item in items]
unlist_comments

['We',
 'met',
 'ratna',
 'june',
 '2012',
 'move',
 'calgari',
 'At',
 'time',
 'got',
 'job',
 'somehow',
 'dream',
 'buy',
 'hous',
 'calgari',
 'We',
 'new',
 'town',
 'look',
 'someon',
 'help',
 'us',
 'buy',
 'hous',
 'We',
 'alreadi',
 'rbc',
 'So',
 'gave',
 'call',
 'custom',
 'servic',
 'refer',
 'ratna',
 'name',
 'us',
 'We',
 'met',
 'person',
 'express',
 'desir',
 'buy',
 'new',
 'hous',
 'she',
 'nice',
 'ask',
 'four',
 'pay',
 'stub',
 'process',
 'mortgag',
 'pre',
 'approv',
 'she',
 'say',
 'wait',
 'anoth',
 'three',
 'month',
 'becom',
 'perman',
 'statu',
 'employ',
 'found',
 'staff',
 'extrem',
 'help',
 'have',
 'work',
 'mani',
 'financi',
 'institut',
 'honestli',
 'agre',
 'custom',
 'like',
 'provid',
 'bad',
 'review',
 'take',
 'time',
 'write',
 'good',
 'review',
 'warrant',
 'dealt',
 'royal',
 'bank',
 'past',
 'found',
 'staff',
 'offer',
 'product',
 'suit',
 'need',
 'made',
 'suggest',
 'even',
 'didnt',
 'ask',
 'wealthi',
 'perfect',
 'credit